In [19]:
import numpy as np
import pandas as pd

In [20]:
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer

In [21]:
from google.colab import files
u=files.upload()

Saving train.csv to train (1).csv


In [22]:
df=pd.read_csv('train.csv',usecols=['Age','Fare','Survived'])

In [23]:
df.dropna(inplace=True)

In [24]:
df.head()

,Survived,Age,Fare
0,0,22.0,7.2500
1,1,38.0,71.2833
2,1,26.0,7.9250
3,1,35.0,53.1000
4,0,35.0,8.0500


In [25]:
x=df.iloc[:,1:]
y=df.iloc[:,0]

In [32]:
x_tr,x_te,y_tr,y_te=train_test_split(x,y,test_size=0.2,random_state=42)

In [33]:
x_tr

,Age,Fare
328,31.0,20.5250
73,26.0,14.4542
253,30.0,16.1000
719,33.0,7.7750
666,25.0,13.0000
...,...,...
92,46.0,61.1750
134,25.0,13.0000
337,41.0,134.5000
548,33.0,20.5250


In [34]:
y_tr

328    1
73     0
253    0
719    0
666    0
      ..
92     0
134    0
337    1
548    0
130    0
Name: Survived, Length: 571, dtype: int64

In [35]:
clf=DecisionTreeClassifier()

In [36]:
clf.fit(x_tr,y_tr)
y_pred=clf.predict(x_te)

In [37]:
accuracy_score(y_te,y_pred)#model has 58% accuracy

0.6293706293706294

In [38]:
np.mean(cross_val_score(DecisionTreeClassifier(),x,y,cv=10,scoring='accuracy'))

0.6288732394366197

In [39]:
kbin_age = KBinsDiscretizer(n_bins=15,encode='ordinal',strategy='quantile')
kbin_fare = KBinsDiscretizer(n_bins=15,encode='ordinal',strategy='quantile')

In [40]:
trf = ColumnTransformer([
    ('first',kbin_age,[0]),
    ('second',kbin_fare,[1])
])

In [42]:
X_train_trf = trf.fit_transform(x_tr)
X_test_trf = trf.transform(x_te)

In [43]:
trf.named_transformers_['first'].bin_edges_


array([array([ 0.42,  6.  , 16.  , 19.  , 21.  , 23.  , 25.  , 28.  , 30.  ,
              32.  , 35.  , 38.  , 42.  , 47.  , 54.  , 80.  ])             ],
      dtype=object)

In [44]:
output = pd.DataFrame({
    'age':x_tr['Age'],
    'age_trf':X_train_trf[:,0],
    'fare':x_tr['Fare'],
    'fare_trf':X_train_trf[:,1]
})


In [48]:
output['age_labels'] = pd.cut(x=x_tr['Age'],
                                    bins=trf.named_transformers_['first'].bin_edges_[0].tolist())
output['fare_labels'] = pd.cut(x=x_tr['Fare'],
                                    bins=trf.named_transformers_['second'].bin_edges_[0].tolist())


In [49]:
output.sample(5)


,age,age_trf,fare,fare_trf,age_labels,fare_labels
131,20.0,3.0,7.0500,0.0,"(19.0, 21.0]","(0.0, 7.25]"
585,18.0,2.0,79.6500,13.0,"(16.0, 19.0]","(76.292, 108.9]"
97,23.0,5.0,63.3583,12.0,"(21.0, 23.0]","(51.479, 76.292]"
251,29.0,7.0,10.4625,4.0,"(28.0, 30.0]","(8.158, 10.5]"
617,26.0,6.0,16.1000,7.0,"(25.0, 28.0]","(14.454, 18.75]"


In [51]:
clf = DecisionTreeClassifier()
clf.fit(X_train_trf,y_tr)
y_pred2 = clf.predict(X_test_trf)


In [52]:
accuracy_score(y_te,y_pred2)

0.6363636363636364